<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

## **Proyecto:** Retención de usuarios de la Plataforma Digital

#### El objetivo de este notebook es realizar el análisis EDA del archivo de `..\Detalle_Trx_Monetarias_YYYYMM.csv` con el objeto de ver que datos incorporar en el archivo de `..\Crosseling_ampliado_YYYYMM.parquet`.

In [ ]:
import pandas as pd
import pandas_profiling
import os

DIR = os.path.abspath(r'..\Data\Export') + '\\'

In [ ]:
uc = [
    'TIPO_CANAL',
    'CANAL',
    'GRUPO_OPERACION',
    'TIPO_OPERACION',
    'TIPO_OPERACION2',
    'IMPORTE',
    'ORIGEN',
    'CANAL_RATIO',
    'CLASIF_TRX'
]

tc = dict(zip(uc, [str] * len(uc)))
tc.update({'IMPORTE': float})

In [ ]:
file = DIR + 'Detalle_Trx_Monetarias_202204.csv'
df = pd.read_csv(file, index_col=False, usecols=uc, dtype=tc).sample(200_000) #Tomo una muestra sino pandas_profiling explota

In [ ]:
df.info()

In [ ]:
# Analizamos
df.reset_index(drop=True, inplace=True)
report = pandas_profiling.ProfileReport(df)
report

#### Variables categóricas con menor grado de correlación *CANAL_RATIO* y *GRUPO_OPERACION*

In [ ]:
# Remuevo los outliers del 1er y 3er quantil de Importe a ver que pasa
Q1 = df['IMPORTE'].quantile(0.25)
Q3 = df['IMPORTE'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)

df = df[~((df['IMPORTE'] < (Q1 - 1.5 * IQR)) | (df['IMPORTE'] > (Q3 + 1.5 * IQR)))]
df.shape

In [ ]:
df.sample(20)

In [ ]:
# Analizamos
df.reset_index(drop=True, inplace=True)
report = pandas_profiling.ProfileReport(df)
report

In [ ]:
# Averiguo la conbinaciones de valores del primer par de variables menos correlacionadas
df.groupby(['CANAL_RATIO', 'GRUPO_OPERACION']).size().reset_index(name='Freq')

In [ ]:
# Averiguo la conbinaciones de valores del segundo par de variables menos correlacionadas
df.groupby(['CANAL_RATIO', 'GRUPO_OPERACION', 'TIPO_OPERACION']).size().reset_index(name='Freq')

In [ ]:
# Averiguo como un cliente puede depositar por canal digital
df[((df['CANAL_RATIO']=='DIGITAL') & (df['TIPO_OPERACION']=='DEPOSITOS'))]

In [ ]:
# Averiguo como un cliente puede depositar por canal digital
df[((df['CANAL_RATIO']=='RESTO') & (df['TIPO_OPERACION']=='MONET BO'))]